In [34]:
import numpy as np
import pandas as pd
import keras
from keras.layers import Dense, Input, LSTM, concatenate, Dropout, Conv2D, MaxPool2D, Embedding, Reshape, Conv1D
from keras.models import Sequential, Model
from keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from keras import metrics
from keras import backend as K

In [2]:
categorical = ['주야', '요일', '발생지시도', '발생지시군구', '사고유형_대분류', '사고유형_중분류', '법규위반', 
            '도로형태_대분류', '도로형태', '당사자종별_1당_대분류', '당사자종별_2당_대분류']
numerical = ['사상자수', '사망자수', '중상자수', '경상자수','부상신고자수']

x_train_num = pd.read_csv('./교통사망사고정보/Kor_Train_교통사망사고정보(12.1~17.6).csv',encoding='cp949', usecols=numerical)

x_train_cat = pd.read_csv('./교통사망사고정보/Kor_Train_교통사망사고정보(12.1~17.6).csv',encoding='cp949', usecols=categorical)

x_test_num = pd.read_csv('./test_kor.csv',encoding='cp949', usecols=numerical)

x_test_cat = pd.read_csv('./test_kor.csv',encoding='cp949', usecols=categorical)

In [3]:
all_data = pd.concat((x_test_cat.dropna(), x_train_cat))
# for col in all_data.select_dtypes(include=[np.object]).columns:
#     print(col, all_data[col].unique())

In [4]:
for column in all_data.select_dtypes(include=[np.object]).columns:
    x_train_cat[column] = x_train_cat[column].astype('category', categories = all_data[column].unique())
    x_test_cat[column] = x_test_cat[column].astype('category', categories = all_data[column].unique())

/home/iron/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  
/home/iron/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
x_train_cat = pd.get_dummies(data=x_train_cat)
x_test_cat = pd.get_dummies(data=x_test_cat)

In [6]:
Case1 = ['사망자수','사상자수','경상자수']
Case2 = ['사상자수', '중상자수', '부상신고자수']
Case3 = ['사상자수', '중상자수', '경상자수' ]
Case4 = ['사망자수', '사상자수', '중상자수' ]

In [35]:
# 수치형 데이터 Case 함수

def numeric_case(start, end):
    
    K.clear_session()
    
    # Train Data
    X = x_train_num.drop(columns=['사상자수']).values
    print(len(X[0]))
    
    # Test Data
    X_test = x_test_num.drop(columns=['사상자수']).values
    
    # Label Data
    Y = x_train_num[['사상자수']].values
    print(len(Y[0]))
    
#     X = np.reshape(X, (X.shape[0], X.shape[1], 1))
#     X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    
#     # Model define

#     num_input = Input(shape=( X.shape[1], 1), name='num_input')
#     x = LSTM(512, return_sequences=True)(num_input)
#     x = Dense((int(len(X[0] + len(Y[0])) * 2 /3)), activation='relu')(x)
#     x = Dropout(0.3)(x)
#     x = Dense((int(len(X[0] + len(Y[0])) * 2 /3)), activation='relu')(x)
#     x = Dropout(0.3)(x)
#     x = Dense((int(len(X[0] + len(Y[0])) * 2 /3)), activation='relu')(x)
#     x = Dropout(0.3)(x)
#     num_output = Dense(len(Y[0]), name='num_output')(x)
    
    
    # Model define
    num_input = Input(shape=(len(X[0]),), name='num_input')
    x = Dense((int(len(X[0] + len(Y[0])) * 2 /3)), activation='relu')(num_input)
    x = Dropout(0.3)(x)
    x = Dense((int(len(X[0] + len(Y[0])) * 2 /3)), activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense((int(len(X[0] + len(Y[0])) * 2 /3)), activation='relu')(x)
    x = Dropout(0.3)(x)
    num_output = Dense(len(Y[0]), name='num_output')(x)

    model = Model(inputs=num_input, outputs=num_output)

    model.compile(optimizer='adam',
                  loss='mse',
                  metrics=['accuracy'])

    learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=25, verbose=1, factor=0.5, min_lr=0.00000001)

    callbacks = [
        learning_rate_reduction, # learning_rate를 점차 감소시킴으로서 최적값에 접근하는 방식
        EarlyStopping('val_loss', patience=15)# val_loss이 최적값에서 멀어지는 순간 epoch가 남아도 학습 중지
    ]
    
    history = model.fit(X, Y, epochs=50, batch_size=128, callbacks=callbacks, validation_split=0.2 )
    
    
    # make a prediction
    Y_test = model.predict(X_test[start:end+1])
    
    del model
    
    return Y_test

In [36]:
numeric_case(0,1)

4
1
Train on 20029 samples, validate on 5008 samples
Epoch 1/50
20029/20029 [==============================] - 1s 43us/step - loss: 6.2879 - acc: 0.0000e+00 - val_loss: 7.9028 - val_acc: 0.0000e+00
Epoch 2/50
20029/20029 [==============================] - 0s 19us/step - loss: 5.7712 - acc: 0.0000e+00 - val_loss: 7.4789 - val_acc: 0.0000e+00
Epoch 3/50
20029/20029 [==============================] - 0s 18us/step - loss: 5.4569 - acc: 0.0000e+00 - val_loss: 7.1189 - val_acc: 0.0000e+00
Epoch 4/50
20029/20029 [==============================] - 0s 18us/step - loss: 5.0892 - acc: 0.3807 - val_loss: 6.8135 - val_acc: 0.7292
Epoch 5/50
20029/20029 [==============================] - 0s 19us/step - loss: 4.8234 - acc: 0.7379 - val_loss: 6.5582 - val_acc: 0.7292
Epoch 6/50
20029/20029 [==============================] - 0s 19us/step - loss: 4.6055 - acc: 0.7379 - val_loss: 6.3460 - val_acc: 0.7292
Epoch 7/50
20029/20029 [==============================] - 0s 19us/step - loss: 4.4294 - acc: 0.7379 -

array([[1.6077368],
       [1.6077368]], dtype=float32)